# Assignment 4 
by [Jay Nakum](https://jaynakum.github.io) [20BCP304D]

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
train=pd.read_csv('./datasets/house-prices/train.csv')

In [46]:
test=pd.read_csv('./datasets/house-prices/test.csv')

In [ ]:
train

In [ ]:
train.isna().sum()[train.isna().sum()>0]

Dropping Id and other columns that have too many missing values

In [49]:
train=train.drop(columns = ['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'])
test=test.drop(columns = ['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'])

In [ ]:
train

In [ ]:
test

In [ ]:
numerical_columns=train.describe().columns
train[train.describe().columns]

Filling in Nan values of numerical columns

In [53]:
for i in numerical_columns:
    train[i]=train[i].replace(np.nan,train[i].mean())
for i in numerical_columns[:-1]:
    test[i]=test[i].replace(np.nan,test[i].mean())

In [ ]:
train[train.describe().columns].isnull().any()

In [ ]:
test[test.describe().columns].isnull().any()

## Filling in Nan values of categorical columns

In [56]:
categorical_columns=[i for i in train.columns if train[i].dtype=='object']

In [57]:
for i in categorical_columns:
    train[i]=train[i].fillna(train[i].mode()[0])
    test[i]=test[i].fillna(test[i].mode()[0])

In [ ]:
train[categorical_columns].isna().sum()

In [ ]:
test[categorical_columns].isna().sum()

## Label Encoding

In [60]:
from sklearn.preprocessing import LabelEncoder
en=LabelEncoder()

In [61]:
for i in categorical_columns:
    train[i]=en.fit_transform(train[i])
    test[i]=en.fit_transform(test[i])

In [ ]:
train[categorical_columns]

## Feature Selection

In [63]:
from feature_engine.selection import DropConstantFeatures,DropDuplicateFeatures,SmartCorrelatedSelection
from sklearn.pipeline import Pipeline

In [64]:
from sklearn.ensemble import RandomForestRegressor
RR=RandomForestRegressor(random_state=0)

In [65]:
pipe=Pipeline([('constant',DropConstantFeatures(tol=0.95)),('duplicate',DropDuplicateFeatures()),('smartcorrelation',SmartCorrelatedSelection(threshold=0.95,estimator=RandomForestRegressor))])

In [66]:
x=train.drop('SalePrice',axis=1)
y=train['SalePrice']

In [67]:
trainnew=pipe.fit_transform(x)
testnew=pipe.fit_transform(test)

## Data Normalization

In [68]:
x=(x-x.mean())/x.std()

In [69]:
RR.fit(x,y)

RandomForestRegressor(random_state=0)

In [70]:
pred_test=RR.predict(test)
pred_train=RR.predict(x)

In [71]:
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error

In [72]:
r2_score(y,pred_train)

0.9812454021001051

In [73]:
mean_absolute_error(y,pred_train)

6454.981924657533

In [74]:
np.sqrt(mean_squared_error(y,pred_train))

10875.719928256385

In [ ]:
print(pd.DataFrame(y.values,pred_train))

In [ ]:
print(pd.DataFrame(pred_test))